In [1]:
import numpy as np
import xarray as xr
import s3fs
import gcsfs
import intake
import dask
#from cmip6_preprocessing.preprocessing import combined_preprocessing

In [2]:
# From Google Cloud:
col = intake.open_esm_datastore("https://storage.googleapis.com/cmip6/pangeo-cmip6.json")

print("col:", col)
print("col_df:", col.df.head())

col: <pangeo-cmip6 catalog with 7674 dataset(s) from 514818 asset(s)>
col_df:   activity_id institution_id     source_id       experiment_id member_id  \
0  HighResMIP           CMCC  CMCC-CM2-HR4  highresSST-present  r1i1p1f1   
1  HighResMIP           CMCC  CMCC-CM2-HR4  highresSST-present  r1i1p1f1   
2  HighResMIP           CMCC  CMCC-CM2-HR4  highresSST-present  r1i1p1f1   
3  HighResMIP           CMCC  CMCC-CM2-HR4  highresSST-present  r1i1p1f1   
4  HighResMIP           CMCC  CMCC-CM2-HR4  highresSST-present  r1i1p1f1   

  table_id variable_id grid_label  \
0     Amon          ps         gn   
1     Amon        rsds         gn   
2     Amon        rlus         gn   
3     Amon        rlds         gn   
4     Amon         psl         gn   

                                              zstore  dcpp_init_year   version  
0  gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...             NaN  20170706  
1  gs://cmip6/CMIP6/HighResMIP/CMCC/CMCC-CM2-HR4/...             NaN  20170706  

In [3]:
### Searching for datasets ###

# Form query dictionary
query = dict(experiment_id = 'historical', variable_id = 'tas', table_id = 'Amon', source_id = 'EC-Earth3')

In [4]:
# Subset catalog and get some metrics grouped by 'source_id'
col_subset = col.search(require_all_on=['source_id'], **query) 
col_subset.df.groupby('source_id')[['experiment_id', 'variable_id', 'table_id']].nunique() # Ordering CSV outut file to display results in this order (column-wise)

,experiment_id,variable_id,table_id
source_id,,,
EC-Earth3,1,1,1


In [5]:
# Output all metadata to CSV
col_subset.df.to_csv(path_or_buf='/home/jovyan/output.csv', index=False)

In [6]:
# Read in data - assign into a dictionary
with dask.config.set(**{'array.slicing.split_large_chunks': False}): # Need this line to silence array size warning
    dsets = col_subset.to_dataset_dict(zarr_kwargs={'consolidated': True},
                                   storage_options={'token': 'anon'})


--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'


In [9]:
# list all merged datasets
[key for key in dsets.keys()]

ds_hist = dsets['CMIP.EC-Earth-Consortium.EC-Earth3.historical.Amon.gr']
print("ds_hist:", ds_hist)

tas_hist = ds_hist['tas'][0] # We index the first available ensemble member
print("tas_hist:", tas_hist.shape)

ds_hist: <xarray.Dataset>
Dimensions:    (lat: 514, lon: 512, time: 1981, bnds: 2, member_id: 73)
Coordinates:
  * lat        (lat) float64 -89.46 -89.46 -88.77 -88.77 ... 88.77 89.46 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * time       (time) datetime64[ns] 1849-12-16T12:00:00 ... 2014-12-16T12:00:00
    height     float64 ...
    lat_bnds   (lat, bnds) float64 dask.array<chunksize=(514, 2), meta=np.ndarray>
    lon_bnds   (lon, bnds) float64 dask.array<chunksize=(512, 2), meta=np.ndarray>
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1981, 2), meta=np.ndarray>
  * member_id  (member_id) <U10 'r9i1p1f1' 'r11i1p1f1' ... 'r25i1p1f1'
Dimensions without coordinates: bnds
Data variables:
    tas        (member_id, time, lat, lon) float32 dask.array<chunksize=(1, 126, 514, 512), meta=np.ndarray>
Attributes: (12/55)
    title:                     EC-Earth3 output prepared for CMIP6
    branch_time_in_child:      0.0
    varia

In [8]:
### 1985-2014 ###
tas_hist = tas_hist.sel(time=slice("1985-01-01", "2014-12-31")) # Index last 30 years using Xarray function
print("tas_hist:", tas_hist.shape, tas_hist)

tas_hist: (360, 514, 512) <xarray.DataArray 'tas' (time: 360, lat: 514, lon: 512)>
dask.array<getitem, shape=(360, 514, 512), dtype=float32, chunksize=(69, 514, 512), chunktype=numpy.ndarray>
Coordinates:
  * lat        (lat) float64 -89.46 -89.46 -88.77 -88.77 ... 88.77 89.46 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
  * time       (time) datetime64[ns] 1985-01-16T12:00:00 ... 2014-12-16T12:00:00
    height     float64 ...
    member_id  <U10 'r9i1p1f1'
Attributes:
    cell_measures:  area: areacella
    cell_methods:   area: time: mean
    comment:        near-surface (usually, 2 meter) air temperature
    history:        2019-07-04T00:44:59Z altered by CMOR: Treated scalar dime...
    long_name:      Near-Surface Air Temperature
    standard_name:  air_temperature
    units:          K
